Imports

In [4]:
import boto3
from boto3.dynamodb.conditions import Key, Attr
import ipywidgets as widgets
from IPython.display import display, clear_output

Initialization

In [5]:

# Widgets for user inputs
profile_input = widgets.Text(description="AWS Profile:", placeholder="e.g., musarisaas")
table_input = widgets.Text(description="Table Name:", placeholder="e.g., CoreUser")
partition_key_input = widgets.Text(description="Partition Key:", placeholder="e.g., PEPSICO-PRE")
sort_key_prefix_input = widgets.Text(description="Sort Key Prefix (optional):", placeholder="e.g., 2024")
run_button = widgets.Button(description="Run Query", button_style="success")
output = widgets.Output()

Query and count

In [6]:
def run_query(b):
    with output:
        clear_output()
        try:
            # Read inputs
            profile_name = profile_input.value.strip()
            table_name = table_input.value.strip()
            partition_value = partition_key_input.value.strip()
            sort_key_prefix = sort_key_prefix_input.value.strip()

            # Initialize boto3 session and DynamoDB table
            session = boto3.Session(profile_name=profile_name)
            dynamodb = session.resource('dynamodb')
            table = dynamodb.Table(table_name)

            # Initialize pagination vars
            record_count = 0
            last_evaluated_key = None

            # Prepare KeyConditionExpression
            key_cond = Key('SupplierID').eq(partition_value)
            if sort_key_prefix:
                key_cond = key_cond & Key('Version').begins_with(sort_key_prefix)

            # Paginate through results
            while True:
                if last_evaluated_key:
                    response = table.query(
                        KeyConditionExpression=key_cond,
                        ExclusiveStartKey=last_evaluated_key
                    )
                else:
                    response = table.query(
                        KeyConditionExpression=key_cond
                    )

                record_count += response['Count']

                # Print fetched items for this page
                for item in response.get('Items', []):
                    print(item)

                last_evaluated_key = response.get('LastEvaluatedKey')
                if not last_evaluated_key:
                    break

            print(f"\nTotal number of records fetched: {record_count}")

        except Exception as e:
            print(f"Error: {e}")

run_button.on_click(run_query)

In [ ]:
# Display widgets and output area
display(widgets.HTML("<b>Enter DynamoDB Query Parameters:</b>"))
display(profile_input, table_input, partition_key_input, sort_key_prefix_input, run_button, output)